In [1]:
from flask import Flask, flash , redirect, render_template , request, session, abort , Markup
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
from keras import backend as K
from werkzeug import secure_filename
import json
from scipy.stats import kurtosis, skew
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from werkzeug import secure_filename

Using TensorFlow backend.


In [2]:
app = Flask(__name__)
app.secret_key = os.urandom(12)

dropdown_list = []
dropdown_list_2 = []

In [3]:
UPLOAD_FOLDER = './Uploads'
ALLOWED_EXTENSIONS = set(['csv'])
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [4]:
@app.route('/upload')
def upload_file():
    dropdown_list.clear()
    dropdown_list_2.clear()
    return render_template('upload.html')

In [5]:
@app.route('/uploader', methods = ['GET', 'POST'])
def uploader_file():
     if request.method == 'POST':
        
        if 'file' not in request.files:
            flash('No file part')
            print('No file part')
            return redirect(request.url)
        file = request.files['file']
        print(file.filename)
        if file.filename == '':
            flash('No selected file')
            print('No selected file')
            return redirect(request.url)
        if file :
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            fpath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            uploadFileAndPredict(fpath)
            with open(fpath) as file:
                allRead = csv.reader(file, delimiter=',')
                lineCount = 0
                for row in allRead:
                    if lineCount==0:
                        lineCount=lineCount+1
                    else:
                        lineCount=lineCount+1
                        dropdown_list_2.append((row[0]))
            return render_template('Result.html',  dropdown_list_2=dropdown_list_2)



In [6]:
@app.route('/input_percent' , methods = ['GET','POST'])
def input_num():
    x = request.form["in"]
    fpath = os.path.join("default", "output.csv")
    line = pd.read_csv(fpath).shape[0]
    y = round((float(x)*line)/100)
    print(line)
    print(y)
    ls = []
    lschurn=[]
    with open(fpath) as file:
        allRead = csv.reader(file, delimiter=',')
        lineCount = 0
        for row in allRead:
            if lineCount == 0:
                lineCount += 1
            elif lineCount <= y and lineCount != 0:
                ls.append(row[0])
                lschurn.append(row[16])
                lineCount += 1
    lss=list(map(lambda x: float(x*100),list(pd.read_csv(fpath)['Approved'][:y].copy())))  
    print(lss)
    print(lschurn)
    return render_template('Percent.html', outList = ls, value_list=lschurn,values_res=lss )


In [7]:
@app.route('/check/<string:dropdown>',methods=['POST','GET'])
def specific(dropdown):
    x = dropdown
    x = search_default(x)
    #Key,Male,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,
    #CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved

    key  = x[0]
    Male = x[1]
    Age = x[2]
    Married  = x[3]
    Debt = x[4]
    BankCustomer  = x[5]
    EducationLevel  = x[6]
    Ethnicity = x[7]
    YearsEmployed  = x[8]
    PriorDefault = x[9]
    Employed = x[10]
    CreditScore = x[11]
    Income = x[15]    
    x = x[16]
    pred= float(x)*100
    values = [pred]
    x = float(x)*100
    x = round(x,2)
    
    return render_template('Chart.html', key=key, 
                           Male=Male, Age=Age, Married=Married, Debt=Debt, 
                           BankCustomer=BankCustomer, EducationLevel=EducationLevel, Ethnicity=Ethnicity,
                           YearsEmployed=YearsEmployed, PriorDefault=PriorDefault, 
                           Employed=Employed, CreditScore = CreditScore,
                           Income = Income, values = values,pred=x)

In [8]:
def preprocess_data(fileInput):
    train = pd.read_csv(fileInput)  
    train.replace('?', np.NaN, inplace = True)
    train['YearsEmployed'] = [x*100 for x in train['YearsEmployed']]
    train['Age'] = train['Age'].astype('float64')
    train['Age'].fillna((train['Age'].mean()), inplace=True)
    train['Married'] = train['Married'].astype('category')
    train['BankCustomer'] = train['BankCustomer'].astype('category')
    train['EducationLevel'] = train['EducationLevel'].astype('category')
    train['Ethnicity'] = train['Ethnicity'].astype('category')
    train['PriorDefault'] = train['PriorDefault'].astype('category')
    train['Employed'] = train['Employed'].astype('category')
    train['DriversLicense'] = train['DriversLicense'].astype('category')
    train['Citizen'] = train['Citizen'].astype('category')

    cat_columns = train.select_dtypes(['category']).columns
    train[cat_columns] = train[cat_columns].apply(lambda x: x.cat.codes)
    train = train.drop(['ZipCode','Male'], axis=1)
    X_test = train.iloc[:,1:].values

    return X_test
    
    

In [9]:
@app.route('/')
def home():
        return render_template('upload.html')


In [10]:
def uploadFileAndPredict(filename):
    K.clear_session()
    dropdown_list_2.clear()
    proceseed_data = preprocess_data(filename)
    model = pickle.load(open('LRModel.pkl', 'rb'))
    y_pred = model.predict_proba(proceseed_data)
    df = pd.read_csv(filename)
    df['Approved'] = [x[0] for x in y_pred]
    df.set_index('Key', inplace=True)
    df.sort_values('Approved', ascending=False, inplace=True)
    fpathr = os.path.join("default", "output.csv")
    df.to_csv(fpathr)  
    return y_pred
             
    

In [11]:
@app.route('/defaultfile', methods = ['GET', 'POST'])
def uploader_default_file():
    fpath = os.path.join("default", "test.csv")
    uploadFileAndPredict(fpath)
    with open(fpath) as file:
        allRead = csv.reader(file, delimiter=',')
        lineCount = 0
        for row in allRead:
            if lineCount==0:
                lineCount=lineCount+1
            else:
                lineCount=lineCount+1
                dropdown_list_2.append((row[0]))
    return render_template('Result.html',  dropdown_list_2=dropdown_list_2)



In [12]:
@app.route('/check_default/<string:dropdown_2>',methods=['POST','GET'])
def specific_default(dropdown_2):
    x = dropdown_2
    
    x = search_default(x)
    key  = x[0]
    Male = x[1]
    Age = x[2]
    Married  = x[3]
    Debt = x[4]
    BankCustomer  = x[5]
    EducationLevel  = x[6]
    Ethnicity = x[7]
    YearsEmployed  = x[8]
    PriorDefault = x[9]
    Employed = x[10]
    CreditScore = x[11]
    Income = x[15]    
    x = x[16]
    pred= float(x)*100
    values = [pred]
    x = float(x)*100
    x = round(x,2)
    
    return render_template('Chart.html', key=key, 
                           Male=Male, Age=Age, Married=Married, Debt=Debt, 
                           BankCustomer=BankCustomer, EducationLevel=EducationLevel, Ethnicity=Ethnicity,
                           YearsEmployed=YearsEmployed, PriorDefault=PriorDefault, 
                           Employed=Employed, CreditScore = CreditScore,
                           Income = Income, values = values,pred=x)

In [13]:
def search_default(cid):
    fpathr = os.path.join("default", "output.csv")
    with open(fpathr) as file:
        allRead = csv.reader(file, delimiter=',')
        for row in allRead:
            if row[0]==cid:
                return row

In [ ]:
if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I1012 13:46:51.095193 140348450322176 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
W1012 13:47:24.375957 140347231078144 deprecation_wrapper.py:119] From /home/redwuie/miniconda3/envs/py3.7.3_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W1012 13:47:24.377744 140347231078144 deprecation_wrapper.py:119] From /home/redwuie/miniconda3/envs/py3.7.3_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



test.csv


W1012 13:47:25.228101 140347231078144 deprecation_wrapper.py:119] From /home/redwuie/miniconda3/envs/py3.7.3_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

I1012 13:47:27.084896 140347231078144 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 13:47:27] "POST /uploader HTTP/1.1" 200 -
I1012 13:48:34.429497 140347231078144 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 13:48:34] "GET / HTTP/1.1" 200 -
I1012 13:49:29.987305 140347231078144 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 13:49:29] "POST /uploader HTTP/1.1" 200 -


test.csv


I1012 13:50:16.881827 140347231078144 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 13:50:16] "GET /check_default/33 HTTP/1.1" 200 -
I1012 16:01:36.975998 140347231078144 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:01:36] "GET / HTTP/1.1" 200 -
I1012 16:01:41.831030 140347231078144 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:01:41] "GET /favicon.ico HTTP/1.1" 404 -
I1012 16:17:07.407591 140347210483456 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:17:07] "GET /check_default/33 HTTP/1.1" 200 -
I1012 16:17:08.044367 140347193698048 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:17:08] "GET /static/js/Chart.min.js HTTP/1.1" 304 -
I1012 16:17:08.047583 140346847655680 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:17:08] "GET /static/js/RGraph.common.annotate.js HTTP/1.1" 304 -
I1012 16:17:08.050226 140346839262976 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:17:08] "GET /static/js/RGraph.bar.js HTTP/1.1" 304 -
I1012 16:17:08.054232 140347202090752 _internal.py:122] 127.0.0.1

34
27
[99.95810125534265, 99.71386765470204, 98.79684309484983, 98.06953457003364, 97.35364296585225, 96.34463676325721, 96.1834165816112, 95.44019902811979, 94.83076073169144, 94.7862207369484, 94.16669134053288, 92.3203562525762, 91.66834404457408, 91.13927805268612, 90.48102637632032, 90.44836379048148, 87.83367981418209, 87.63909584507238, 86.707304806014, 85.45338025409819, 83.3815684840822, 81.30211894930449, 74.98905851526655, 70.93538527404844, 70.22111019855343, 60.01734967415866, 32.16703647812104]
['0.9995810125534266', '0.9971386765470204', '0.9879684309484985', '0.9806953457003363', '0.9735364296585223', '0.9634463676325722', '0.961834165816112', '0.954401990281198', '0.9483076073169145', '0.9478622073694841', '0.9416669134053289', '0.923203562525762', '0.9166834404457408', '0.9113927805268613', '0.9048102637632031', '0.9044836379048149', '0.8783367981418209', '0.8763909584507238', '0.86707304806014', '0.8545338025409819', '0.8338156848408219', '0.8130211894930448', '0.749

I1012 16:18:24.754762 140347193698048 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:18:24] "POST /input_percent HTTP/1.1" 200 -
I1012 16:18:24.840492 140347193698048 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:18:24] "GET /static/js/RGraph.common.tooltips.js HTTP/1.1" 200 -
I1012 16:18:24.849638 140346847655680 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:18:24] "GET /static/js/RGraph.hbar.js HTTP/1.1" 200 -
I1012 16:18:24.851135 140347210483456 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:18:24] "GET /static/js/RGraph.common.key.js HTTP/1.1" 200 -
I1012 16:18:33.200440 140346847655680 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 16:18:33] "GET /check/9 HTTP/1.1" 200 -
I1012 17:01:45.249345 140346847655680 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 17:01:45] "GET / HTTP/1.1" 200 -
I1012 17:01:46.227098 140346847655680 _internal.py:122] 127.0.0.1 - - [12/Oct/2019 17:01:46] "GET /favicon.ico HTTP/1.1" 404 -
